In [1]:
# 1. Get 10,000 feeds - Assignment 5

In [60]:
import sys
import requests
calais_url = 'https://api.thomsonreuters.com/permid/calais'

import webhoseio, os
from gensim.models import KeyedVectors
import json
from scipy import spatial
from simhash import Simhash, SimhashIndex
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer

stopwords = set(nltk.corpus.stopwords.words('english'))

import spacy
import en_core_web_sm

In [61]:
def sendFile(input_data, headers, output_dir='./'):
#     with open(file_name, 'rb') as input_data:
    response = requests.post(calais_url, data=input_data, headers=headers, timeout=80)
    print ('status code: %s' % response.status_code)
    content = response.text
#     print ('Results received: %s' % content)
    if response.status_code == 200:
        saveFile('tesla_text', output_dir, content)
        return response
            
def saveFile(file_name, output_dir, content):
    output_file_name = os.path.basename(file_name) + '.json'
    output_file = open(os.path.join(output_dir, output_file_name), 'wb')
    output_file.write(content.encode('utf-8'))
    output_file.close()

In [76]:
print(feeds[11]['title'])
access_token = 'ZR2BvCL0pxKthAeKa0FKPbb9I9At9ows'
headers = {'X-AG-Access-Token' : access_token, 
           'Content-Type' : 'text/raw', 
           'outputformat' : 'application/json'
          }
input_data = feeds[11]['title'].encode('utf-8')
response = sendFile(input_data, headers)
tagging_result = json.loads(response.content)

NPR News: Microsoft Urges Congress To Regulate Facial Recognition Technology
status code: 200


In [77]:
def extract_entity(tagging_result):
    for hash in tagging_result:        
        if '_typeGroup' in tagging_result[hash] and tagging_result[hash]['_typeGroup'] == 'entities':
#             print(tagging_result[hash])
            print(tagging_result[hash]['_type'], ':', tagging_result[hash]['name'], ',', tagging_result[hash]['relevance'])
extract_entity(tagging_result)

Technology : Facial Recognition Technology , 0.8
Organization : Congress , 0.8
Company : Microsoft , 0.8


In [66]:
webhoseio.config(token="2c45808b-852d-4156-af2f-faf960992ea9")
query_params = {"q": "organization:google language:english", #change to other org to get 10,000 feeds
                "sort": "crawled"}
output = webhoseio.query("filterWebContent", query_params)
print(output['posts'][0]['text']) # Print the text of the first post
print(output['posts'][0]['published']) # Print the text of the first post publication date

# Print the site of the first post
print(output['posts'][0]['thread']['site'])

feeds = []
for item in output['posts']:
    feeds.append(item)
    #print(item['title'])
print(len(feeds))

count = 99
while count > 0:    
    output = webhoseio.get_next()
    for item in output['posts']:
        feeds.append(item)
    count -= 1
#     if len(feeds) > 10000:
#         break
# feeds = feeds[:10000]
print(len(feeds)) #try changed org to get 10,000 feeds

import json
with open("anly540_final.json", "w") as myfile: #change file name to other org
    for feed in feeds:
        line = json.dumps(feed)
        myfile.write(line)
        myfile.write("\n")

Reekado Banks Celebrates 25th Birthday With 25 Customized Cakes (Photos) Thursday, 6 December 2018
Nigerian singer and songwriter, Ayoleyi Hanniel Solomon, popularly known by his stage name, Reekado Banks, turned a year older today, and is celebrating his birthday in style.
The artiste who is currently signed to Don Jazzy’s Mavin Records, took to his Instagram page to reveal that his brother and wife came to celebrate the birthday with him and brought him 25 customized cakes.
He shared a photo while posing in front of the cakes and wrote; “So my brother and his family brought 25 cakes for my birthday. Who wants cake, please? Thank you @thistemi @de_mure_ @thefirefunmi.”
See another photo of the birthday boy and his cakes below;
Stay updated with the latest Nigerian news from Information Nigeria
2018-12-06T23:50:00.000+02:00
newsfetchers.com
100
10000


In [ ]:
# 2. 

In [5]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [47]:
model_path = './'
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin', True)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [48]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [49]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [50]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

In [5]:
json_data = open("anly540_final.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))
print(len(feeds_read_from_file))

for feed in feeds_read_from_file[:10]:
    print(feed['title'], '\n')
    print(feed['text'], '\n')
    print(feed['url'], '\n')
#print(feeds_read_from_file[0]['title'])

10000
Reekado Banks Celebrates 25th Birthday With 25 Customized Cakes (Photos) 

Reekado Banks Celebrates 25th Birthday With 25 Customized Cakes (Photos) Thursday, 6 December 2018
Nigerian singer and songwriter, Ayoleyi Hanniel Solomon, popularly known by his stage name, Reekado Banks, turned a year older today, and is celebrating his birthday in style.
The artiste who is currently signed to Don Jazzy’s Mavin Records, took to his Instagram page to reveal that his brother and wife came to celebrate the birthday with him and brought him 25 customized cakes.
He shared a photo while posing in front of the cakes and wrote; “So my brother and his family brought 25 cakes for my birthday. Who wants cake, please? Thank you @thistemi @de_mure_ @thefirefunmi.”
See another photo of the birthday boy and his cakes below;
Stay updated with the latest Nigerian news from Information Nigeria 

http://omgili.com/ri/jHIAmI4hxg_gnkhwwu0uPS8yWYINtJ0Pto3iBG_E36FJvDsLpWdwhZILHof3pXolyWQSe3Dz7OBic5zEIxhjh1HeKH

In [6]:
# Original set of Titles and Publish dates
feeds = []
i = 0
for feed in feeds_read_from_file:
    feed['id'] = i
#     if feed['language'] == 'english':
#         print(feed['language'])    
#     print(str(feed['id']), str(feed['title']),  feed['published'])
    i += 1
    feeds.append(feed)

In [198]:
distance = 10
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds]
index = SimhashIndex(objs, k=distance)
index.bucket_size()

5986

In [ ]:
dups = set()
for feed in feeds:
    # Select a feed from the list based on its index
    SELECTED_INDEX = feed['id'] #27
    if SELECTED_INDEX in dups:
        continue
    feed_sel = feeds[SELECTED_INDEX]
    print(feed_sel['title'])
    # calculate  hash value
    feed_hash = Simhash(str(feed_sel['title']))
    # find all duplicate indices
    dup_indices = index.get_near_dups(feed_hash)
    for dup_index in dup_indices:
        if dup_index != str(SELECTED_INDEX):
            dups.add(dup_index)
#             print(feeds[int(dup_index)]['title'])
#     print(dups)
    print("Number of duplicates (SimHash): " + str(len(dup_indices)))
len(dups)

Big bucket found. key:1:0, len:301
Big bucket found. key:14:1, len:346
Big bucket found. key:16:2, len:204
Big bucket found. key:17:3, len:245
Big bucket found. key:15:4, len:326
Big bucket found. key:8:5, len:333
Big bucket found. key:18:6, len:227
Big bucket found. key:6:7, len:442
Big bucket found. key:8:8, len:344
Big bucket found. key:10:0, len:454
Big bucket found. key:10:1, len:427
Big bucket found. key:12:2, len:278
Big bucket found. key:3:3, len:201
Big bucket found. key:7:4, len:275
Big bucket found. key:11:5, len:275
Big bucket found. key:13:6, len:364
Big bucket found. key:7:7, len:343


Reekado Banks Celebrates 25th Birthday With 25 Customized Cakes (Photos)
Number of duplicates (SimHash): 1
Google’s giant “kite” can generate wind energy from almost anywhere


Big bucket found. key:1a:8, len:207
Big bucket found. key:0:9, len:642
Big bucket found. key:1c:0, len:580
Big bucket found. key:19:1, len:233
Big bucket found. key:0:2, len:989
Big bucket found. key:f:3, len:308
Big bucket found. key:e:4, len:304
Big bucket found. key:c:5, len:368
Big bucket found. key:14:6, len:444
Big bucket found. key:16:7, len:586
Big bucket found. key:10:8, len:493
Big bucket found. key:a:9, len:514
Big bucket found. key:18:0, len:706
Big bucket found. key:10:1, len:427
Big bucket found. key:11:2, len:378
Big bucket found. key:b:3, len:216
Big bucket found. key:1d:4, len:259
Big bucket found. key:c:5, len:368
Big bucket found. key:8:6, len:271
Big bucket found. key:5:7, len:216
Big bucket found. key:1:8, len:271
Big bucket found. key:a:9, len:514


Number of duplicates (SimHash): 1
Google Alert - testing
Number of duplicates (SimHash): 5
Press conference for Mohammed Bin Salman
Number of duplicates (SimHash): 1

Big bucket found. key:8:0, len:313
Big bucket found. key:11:3, len:241
Big bucket found. key:1:4, len:220
Big bucket found. key:1d:5, len:344
Big bucket found. key:14:7, len:272
Big bucket found. key:d:8, len:267
Big bucket found. key:f:9, len:229
Big bucket found. key:5:0, len:297
Big bucket found. key:7:1, len:300
Big bucket found. key:10:2, len:475
Big bucket found. key:1c:3, len:412
Big bucket found. key:1:4, len:220
Big bucket found. key:8:5, len:333
Big bucket found. key:e:6, len:239
Big bucket found. key:3:8, len:293
Big bucket found. key:18:0, len:706
Big bucket found. key:a:1, len:273
Big bucket found. key:0:2, len:989
Big bucket found. key:14:3, len:448


...

Big bucket found. key:17:3, len:245
Big bucket found. key:2:4, len:281
Big bucket found. key:c:5, len:368
Big bucket found. key:e:6, len:239
Big bucket found. key:a:7, len:344
Big bucket found. key:14:8, len:602
Big bucket found. key:10:9, len:327
Big bucket found. key:16:0, len:255
Big bucket found. key:14:1, len:346
Big bucket found. key:9:2, len:335
Big bucket found. key:e:3, len:305
Big bucket found. key:17:4, len:337
Big bucket found. key:12:5, len:213
Big bucket found. key:4:6, len:310
Big bucket found. key:0:7, len:248
Big bucket found. key:9:8, len:298
Big bucket found. key:18:9, len:294
Big bucket found. key:0:0, len:402
Big bucket found. key:16:3, len:356
Big bucket found. key:1f:4, len:276
Big bucket found. key:12:5, len:213
Big bucket found. key:7:8, len:283
Big bucket found. key:2:9, len:418
Big bucket found. key:1c:0, len:580
Big bucket found. key:6:1, len:410
Big bucket found. key:0:2, len:989


Number of duplicates (SimHash): 1
การเมืองไม่ขัดแย้ง สร้างแรงกระตุ้นเศรษฐกิจ มีคุณภาพชีวิตที่ดี : พรรคชาติไทยพัฒนา Chartthaipattana Party | pornthep_showhuay
Number of duplicates (SimHash): 1
Site Build and testimonial Tea 4 You
Number of duplicates (SimHash): 1
Google Alert - Series


Big bucket found. key:5:3, len:305
Big bucket found. key:4:4, len:480
Big bucket found. key:9:5, len:458
Big bucket found. key:15:6, len:805
Big bucket found. key:7:7, len:343
Big bucket found. key:16:8, len:343
Big bucket found. key:9:9, len:590
Big bucket found. key:14:0, len:271
Big bucket found. key:15:1, len:318
Big bucket found. key:12:3, len:318
Big bucket found. key:7:4, len:275
Big bucket found. key:c:5, len:368
Big bucket found. key:15:6, len:805
Big bucket found. key:18:7, len:239
Big bucket found. key:3:8, len:293
Big bucket found. key:1b:9, len:254
Big bucket found. key:1c:0, len:580
Big bucket found. key:0:1, len:503
Big bucket found. key:14:3, len:448
Big bucket found. key:7:4, len:275
Big bucket found. key:c:5, len:368
Big bucket found. key:14:6, len:444
Big bucket found. key:1f:8, len:225
Big bucket found. key:2:9, len:418
Big bucket found. key:b:0, len:211
Big bucket found. key:18:1, len:296
Big bucket found. key:0:2, len:989
Big bucket found. key:16:3, len:356


Number of duplicates (SimHash): 19
Pivotal announces new serverless framework
Number of duplicates (SimHash): 6
We Love the Internet 2018/50: The Death of Hollywood’s middle class edition
Number of duplicates (SimHash): 1
Tributes paid to former Music Week editor David Dalton and publisher Tony Evans


Big bucket found. key:a:4, len:291
Big bucket found. key:5:5, len:617
Big bucket found. key:11:6, len:343
Big bucket found. key:1e:7, len:402
Big bucket found. key:18:8, len:273
Big bucket found. key:0:9, len:642
Big bucket found. key:17:0, len:214
Big bucket found. key:1:1, len:414
Big bucket found. key:4:2, len:335
Big bucket found. key:c:3, len:413
Big bucket found. key:8:4, len:235
Big bucket found. key:15:6, len:805
Big bucket found. key:2:7, len:440
Big bucket found. key:c:8, len:438


In [201]:
for dupi in dup_indices:
    print(feeds[int(dupi)]['title'])

Curiosity Rover Finds Mysterious Metallic Object on Mars | Mysterious Universe


In [73]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is np.nan:
        result = 0
        
    return result

In [74]:
count_dup = 0
for dupi in dup_indices:
    try:
        score = vec_similarity(feed_sel['title'], feeds[int(dupi)]['title'], model_word2vec)
#         print(score)
    except:
        score = 0
    if score > 0.9:
        count_dup += 1
        print(feeds[int(dupi)]['title'])
print("Number of Dupies (SimHash + WordVectors): " + str(count_dup))

5 Tracks Of Wealth Membership, #Self #Help
Number of Dupies (SimHash + WordVectors): 1


In [203]:
# Save as txt file and print deduplicated subset of id, titles and published dates
# dup_indices_set = set(dup_indices)
with open('./deduped.txt', 'w') as f:
    for feed in feeds:
        if str(feed['id']) not in dups:
            print(' '.join((str(feed['id']), str(feed['title']),  feed['published'])))
            f.write(' '.join((str(feed['id']), str(feed['title']),  feed['published'], '\n')))

0 Reekado Banks Celebrates 25th Birthday With 25 Customized Cakes (Photos) 2018-12-06T23:50:00.000+02:00
1 Google’s giant “kite” can generate wind energy from almost anywhere 2018-12-07T01:24:00.000+02:00
2 Google Alert - testing 2018-12-07T01:25:00.000+02:00
3 Press conference for Mohammed Bin Salman 2018-12-05T21:44:00.000+02:00
4 He comes with his own red carpet! 2018-12-03T15:50:00.000+02:00
5 What’s At The Bottom Of The Great Blue Hole? 2018-12-05T13:01:00.000+02:00
6 Google Alert - target 2018-12-07T01:26:00.000+02:00
7 Worker’s residence building gutted by fire 2018-12-06T21:33:00.000+02:00
8 Woman ‘steals blanket’ from stray cat and strikes the cat with the blanket 2018-12-06T23:19:00.000+02:00
10 Spangenberg named Google Cloud Academic All-America® of the Year for DI men’s soccer 2018-12-06T12:00:00.000+02:00
12 Stone tools date early humans in North Africa to 2.4 million years ago 2018-12-03T14:42:00.000+02:00
13 Google is Adding Personalized Audio News Feeds to Assistant - S

1832 Stop Misusing Science To Bolster Religious Fables 2018-12-06T22:28:00.000+02:00
1833 Get your application optimized by the mobile app development agency to increase its installations 2018-12-07T08:04:00.000+02:00
1835 4 Ways to Use Marketing Push Techniques to Scale Up Your Business 2018-12-06T18:00:00.000+02:00
1836 SMILE 2018-12-07T07:04:00.000+02:00
1837 The Best of Captain Capitalism Trio 2018-12-07T08:04:00.000+02:00
1838 How to utilize the controls on the Google email propelled settings page? 2018-12-07T02:00:00.000+02:00
1841 Top 5 Places To Visit In Meghalaya 2018-12-06T14:50:00.000+02:00
1842 Google Maps Update: How Smart Recommendations Impact Your Business 2018-12-07T07:19:00.000+02:00
1843 Top 5 Places To Visit In Orrisa 2018-12-06T13:03:00.000+02:00
1845 Henry Kissinger warns tech, political leaders about unregulated AI 2018-12-07T09:09:00.000+02:00
1857 #1yrago (Virtually) No one should ever own an Echo or any other "voice assistant" product 2018-12-07T07:41:00.000+0

3305 Accelerating a Black Hole 2018-12-07T13:43:00.000+02:00
3307 Fusion methodologies 2018-12-07T13:13:00.000+02:00
3308 Ideal gas and moveable pistons 2018-12-07T13:03:00.000+02:00
3309 Intensity of fringes in a Double Slit experiment 2018-12-07T13:14:00.000+02:00
3313 “Everyone’s for Sale”: A Generation of Digital-Media Darlings Prepares for a Frigid Winter 2018-12-07T03:00:00.000+02:00
3315 What is intuitive or physical meaning of wave functional and field configuration and field eigen function? 2018-12-07T13:00:00.000+02:00
3317 Peakz Stocks in Circle K Stores and Google HQ, Ireland 2018-12-07T12:20:00.000+02:00
3318 Google Pixel 3 XL review 2018-12-07T12:00:00.000+02:00
3319 LCWU’s 8th Convocation, Position Holders Get Gold Medals 2018-12-07T08:13:00.000+02:00
3321 Pixel 3 review 2018-12-07T11:58:00.000+02:00
3323 Downtown Miami offices landing more tech companies 2018-12-07T13:37:00.000+02:00
3325 Медведев щас по телику сказал, 2018-12-06T21:02:00.000+02:00
3326 How To Build a B

4781 The Dark Days Of Facebook, And The Light Ahead 2018-12-07T19:26:00.000+02:00
4782 How to increase page speed to improve SEO results - Search Engine Watch Search Engine Watch 2018-12-07T02:00:00.000+02:00
4784 #Religion, Las Siete Leyes De La Conciencia M 2018-12-07T18:47:00.000+02:00
4785 A Christmas Chronology | 2018-12-07T02:00:00.000+02:00
4786 Record levels of mercury released by thawing permafrost in Canadian Arctic | AGR Daily 60 Second News Bites 2018-12-07T02:00:00.000+02:00
4787 Media Beat: December 07, 2018 2018-12-07T10:36:00.000+02:00
4788 Can somebody give me cracked XEvil program??? 2018-12-07T14:19:00.000+02:00
4793 Daily Simple Questions – ASK AND ANSWER HERE!- December 07 2018-12-07T19:35:00.000+02:00
4795 SEO for my website 2018-12-07T19:26:00.000+02:00
4798 Port a cool garage fan (Grove City) $500 2018-12-07T18:31:00.000+02:00
4800 Should a social media check be required for aspiring gun owners? 2018-12-07T19:16:00.000+02:00
4801 Google Site Kit for WordPress 20

6127 Quotes About Life and Change | Dalai Lama 2018-11-28T14:00:00.000+02:00
6128 Nest Best Green Monday Nest Deals: Save Up to $120 w/ Google Home Hub Bundles 2018-12-07T23:50:00.000+02:00
6129 Fill up your $129 Galaxy Tab E tablet using the free $25 Google Play credit 2018-12-08T00:15:00.000+02:00
6130 Samsung Galaxy S9 64GB Lilac Purple Color -UNLOCKED- (Sioux Falls, S.D) $480 2018-12-07T23:05:00.000+02:00
6131 Xpert 12 DC Volt Electric 1/2" Drive Impact Wrench Gun Working w/ Coil (Norwood) $34 2018-12-07T23:12:00.000+02:00
6132 Rochdale News | News Headlines | Fears over chemicals at warehouse site - Rochdale Online 2018-12-07T02:00:00.000+02:00
6133 How to Succeed in a System Design Interview – Pramp Blog | Coding Interview & Job Search Resources for Developers 2018-12-08T00:22:00.000+02:00
6134 Australia anti-encryption law rushed to passage 2018-12-07T23:13:00.000+02:00
6135 Will Sparks – Monsta – Single [iTunes Plus AAC M4A] (2018) 2018-12-07T23:24:00.000+02:00
6136 Xiaomi Mi n

7468 Period of each state 2018-12-08T05:28:00.000+02:00
7469 A cover of Locally connected space with certain compactness property 2018-12-08T05:25:00.000+02:00
7471 Intuition behind Riemann integrabillity and sequences of partitions 2018-12-08T05:17:00.000+02:00
7472 Happy Birthday Thomas Alan Waits! Thanx Be To The Muses, You Arrived On This Date…… 2018-12-08T01:49:00.000+02:00
7473 The whole concept of dividing by zero? 2018-12-08T05:18:00.000+02:00
7474 If $p$ is prime, show $kn$ mod $p$ cycles through all values $[0, p-1]$, where $k,n$ are integers. 2018-12-08T05:18:00.000+02:00
7475 Why doesn't $\sum \frac{sin(\frac{1}{n})}{\sqrt(n)}$ diverge? 2018-12-08T05:07:00.000+02:00
7476 Update for Liu 2018-12-08T05:27:00.000+02:00
7477 Create a space between clouds 2018-12-08T05:01:00.000+02:00
7478 Does weak convergence imply time average convergence? 2018-12-08T05:26:00.000+02:00
7479 Why is $\int_\Sigma \log \frac{c}{|\cos \theta|}d\sigma 1$? 2018-12-08T04:47:00.000+02:00
7480 Miss Supr

8383 Nkomezi faces up to three weeks out | The New Times | Rwanda 2018-12-07T02:00:00.000+02:00
8384 Tanzania to avoid loans with stringent conditions | The New Times | Rwanda 2018-12-05T02:00:00.000+02:00
8385 Nominees for 2018 Groove awards unveiled | The New Times | Rwanda 2018-12-04T02:00:00.000+02:00
8386 Is fake news here to stay? | The New Times | Rwanda 2018-12-06T02:00:00.000+02:00
8387 Why the Rwigaras were acquitted | The New Times | Rwanda 2018-12-06T02:00:00.000+02:00
8388 Mukura players warned against complacency | The New Times | Rwanda 2018-12-06T02:00:00.000+02:00
8389 Boyfriend jeans; a must have? | The New Times | Rwanda 2018-12-05T02:00:00.000+02:00
8390 Inside the world of ‘blessers’ | The New Times | Rwanda 2018-12-06T02:00:00.000+02:00
8392 Cannock railway station to get makeover 2018-12-08T08:01:00.000+02:00
8393 Fuji Electric Fresh Bean to Cup Coffee Machine Commercial (Los Angeles) $7999 2018-12-08T08:39:00.000+02:00
8394 Friction energy dissipation calculatio

9383 ahoySoyJoy 2018-12-07T05:57:00.000+02:00
9384 They actually think that their costumes have transformed them 2018-12-08T14:27:00.000+02:00
9385 Not My Pic. But, damn, it’s awesome. 2018-12-08T14:19:00.000+02:00
9386 China At Its Best Again 2018-12-08T13:59:00.000+02:00
9387 Spotify is better. Change my mind 2018-12-08T13:40:00.000+02:00
9388 Free Domain + Web Hosting & Create a website REDEMPTION CODE 2018-12-08T14:55:00.000+02:00
9389 Off-duty police officer foils potential moped robbers in Kingston 2018-12-08T13:09:00.000+02:00
9390 Tech: 'F--- you leakers': A former senior Google employee says a frantic quest to stop internal info getting out is now management's 'number one priority' (GOOGL) 2018-12-08T13:48:00.000+02:00
9391 Tech: 'F--- you leakers': A former senior Google employee says a frantic quest to stop internal info getting out is now management's 'number one priority' (GOOGL) 2018-12-08T12:55:00.000+02:00
9392 Mixed Media 2018-12-08T07:54:00.000+02:00
9395 How to make 

In [211]:
f = feeds[1419]['title']
feed_hash = Simhash(str(f))
inds = index.get_near_dups(feed_hash)
for i in inds:
    print(feeds[int(i)]['title'])

Big bucket found. key:2:0, len:236
Big bucket found. key:1a:1, len:214
Big bucket found. key:1a:2, len:204
Big bucket found. key:1f:3, len:368
Big bucket found. key:1f:4, len:276
Big bucket found. key:a:5, len:222
Big bucket found. key:0:6, len:500
Big bucket found. key:12:7, len:401
Big bucket found. key:7:8, len:283
Big bucket found. key:1b:9, len:254


Landing page in existing Wordpress.
Landing page in existing Wordpress.
Landing page in existing Wordpress.
Landing page in existing Wordpress.


In [12]:
def tokenize_titles(title):
    tokens = nltk.word_tokenize(title)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    
    for token in tokens:
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token not in stopwords:
            filtered_tokens.append(token.lower())
    
    lemmas = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]

    return lemmas

In [13]:
def clstr_lda(num_topics, titles):
    # top words to be identified
    n_top_words = 10

    tf_vectorizer = CountVectorizer(max_df=0.92, min_df=0.02, max_features=2000,
                                    tokenizer=tokenize_titles, ngram_range=(4,5))

    tf = tf_vectorizer.fit_transform(titles)

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=1000,
                                    learning_method='batch', learning_offset=10.,
                                    random_state = 1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()

    # print top topic words
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    return topics

In [14]:
feed_titles = []

for feed in feeds:
    feed_titles.append(str(feed['title']))

print("Total number of titles: " + str(len(feed_titles)))

Total number of titles: 10100


In [24]:
feed_titles[14]

'Tech execs to gather at White House on Thursday'

In [26]:
topics = clstr_lda(1000, feed_titles)

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [30]:
nlp = en_core_web_sm.load()
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"] 

In [28]:
for feed in feeds_read_from_file[3:4]:
    selected_feed = feed['text']
    print(selected_feed) 

Facebook's board of directors on Wednesday defended COO Sheryl Sandberg's request to investigate billionaire George Soros, a prominent critic of the social network. The tech giant has acknowledged that Sandberg asked her staff to look into Soros' financial motivations after he called companies like Facebook and Google a "menace" during a speech at the World Economic Forum in Davos, Switzerland.
"As is to be expected following an attack from such a well-known and widely respected figure, Facebook staff immediately initiated research to attempt to understand the motivations driving the criticism, financial or otherwise," the letter says. "To be clear, Ms. Sandberg's question was entirely appropriate given her role as COO. When a well-known and outspoken investor attacks your company publicly, it is fair and appropriate to do this level of diligence."
Facebook General Council Colin Stretch sent the letter Wednesday to Patrick Gaspard, president of Soros' Open Society Foundations, on behal

In [31]:
import re
from nltk import word_tokenize, sent_tokenize, ngrams
sentences = sent_tokenize(selected_feed)
for sentence in sentences:
    print(sentence)

Facebook's board of directors on Wednesday defended COO Sheryl Sandberg's request to investigate billionaire George Soros, a prominent critic of the social network.
The tech giant has acknowledged that Sandberg asked her staff to look into Soros' financial motivations after he called companies like Facebook and Google a "menace" during a speech at the World Economic Forum in Davos, Switzerland.
"As is to be expected following an attack from such a well-known and widely respected figure, Facebook staff immediately initiated research to attempt to understand the motivations driving the criticism, financial or otherwise," the letter says.
"To be clear, Ms. Sandberg's question was entirely appropriate given her role as COO.
When a well-known and outspoken investor attacks your company publicly, it is fair and appropriate to do this level of diligence."
Facebook General Council Colin Stretch sent the letter Wednesday to Patrick Gaspard, president of Soros' Open Society Foundations, on behal

In [32]:
def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

In [33]:
def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

In [34]:
def getVerbsFromConjunctions(verbs):
    moreVerbs = []
    for verb in verbs:
        rightDeps = {tok.lower_ for tok in verb.rights}
        if "and" in rightDeps:
            moreVerbs.extend([tok for tok in verb.rights if tok.pos_ == "VERB"])
            if len(moreVerbs) > 0:
                moreVerbs.extend(getVerbsFromConjunctions(moreVerbs))
    return moreVerbs

In [35]:
def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

In [36]:
def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

In [37]:
def findSVs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs

In [38]:
def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

In [39]:
def getObjsFromAttrs(deps):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(getObjsFromPrepositions(rights))
                    if len(objs) > 0:
                        return v, objs
    return None, None

In [40]:
def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

In [41]:
def getAllSubs(v):
    verbNegated = isNegated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    return subs, verbNegated

In [42]:
def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    #potentialNewVerb, potentialNewObjs = getObjsFromAttrs(rights)
    #if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
    #    objs.extend(potentialNewObjs)
    #    v = potentialNewVerb

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

In [43]:
def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjs(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lower_ if verbNegated or objNegated else v.lower_, obj.lower_))
    return svos

In [44]:
def getAbuserOntoVictimSVOs(tokens):
    maleAbuser = {'he', 'boyfriend', 'bf', 'father', 'dad', 'husband', 'brother', 'man'}
    femaleAbuser = {'she', 'girlfriend', 'gf', 'mother', 'mom', 'wife', 'sister', 'woman'}
    neutralAbuser = {'pastor', 'abuser', 'offender', 'ex', 'x', 'lover', 'church', 'they'}
    victim = {'me', 'sister', 'brother', 'child', 'kid', 'baby', 'friend', 'her', 'him', 'man', 'woman'}

    svos = findSVOs(tokens)
    wnl = WordNetLemmatizer()
    passed = []
    for s, v, o in svos:
        s = wnl.lemmatize(s)
        v = "!" + wnl.lemmatize(v[1:], 'v') if v[0] == "!" else wnl.lemmatize(v, 'v')
        o = "!" + wnl.lemmatize(o[1:]) if o[0] == "!" else wnl.lemmatize(o)
        if s in maleAbuser.union(femaleAbuser).union(neutralAbuser) and o in victim:
            passed.append((s, v, o))
    return passed

In [45]:
def printDeps(toks):
    for tok in toks:
        print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])

def testSVOs():
    #nlp = English()
    for sentence in sentences:
        tok = nlp(sentence)
        svos = findSVOs(tok)
    #     printDeps(tok)
    #    assert set(svos) == {('i', '!have', 'assistance'), ('he', '!provide', 'support')}
        if svos:
            for svo in svos:
                print('S:{0}, V:{1}, O:{2}'.format((svo[0]), svo[1], svo[2]))
        else:
            print('No SVO')

In [46]:
testSVOs()

S:board, V:defended, O:request
S:request, V:investigate, O:soros
S:sandberg, V:asked, O:staff
S:he, V:called, O:companies
S:he, V:called, O:menace
S:staff, V:initiated, O:research
S:motivations, V:driving, O:criticism
No SVO
S:it, V:do, O:level
S:stretch, V:sent, O:letter
S:stretch, V:sent, O:to
S:journal, V:reported, O:news
S:facebook, V:investigate, O:affairs
S:facebook, V:investigate, O:soros
S:that, V:provides, O:research
No SVO
S:we, V:have, O:lots
S:they, V:smear, O:anyone
No SVO
No SVO
S:materials, V:presented, O:me
S:i, V:received, O:number
S:she, V:told, O:him
S:she, V:knew, O:nothing
S:she, V:sent, O:email
No SVO
No SVO
